In [ ]:
# Загружаем файлы c датасетами в Google Colab
!wget 'https://drive.google.com/uc?export=download&id=1q4wMQlDhr1lZUV9iXsJGY7tMSuOXEET6' -O data.zip


--2022-04-02 16:59:13--  https://drive.google.com/uc?export=download&id=1q4wMQlDhr1lZUV9iXsJGY7tMSuOXEET6
Resolving drive.google.com (drive.google.com)... 74.125.31.138, 74.125.31.113, 74.125.31.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.31.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5hakdf32drdap00f2n85gj73ip1icv9t/1648918725000/14904333240138417226/*/1q4wMQlDhr1lZUV9iXsJGY7tMSuOXEET6?e=download [following]
--2022-04-02 16:59:15--  https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5hakdf32drdap00f2n85gj73ip1icv9t/1648918725000/14904333240138417226/*/1q4wMQlDhr1lZUV9iXsJGY7tMSuOXEET6?e=download
Resolving doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.googleusercontent.com)... 74.125.134.132, 2607:f8b0:400c:c00::84
Connecting to doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.goo

In [ ]:
!unzip data.zip


Archive:  data.zip
  inflating: home_page_table.csv     
  inflating: payment_confirmation_table.csv  
  inflating: payment_page_table.csv  
  inflating: search_page_table.csv   
  inflating: user_table.csv          


In [ ]:
# Импортируем в библиотеку Pandas
import pandas as pd

In [ ]:
# Загружаем в Python с помощью Pandas все датасеты
user_table = pd.read_csv('user_table.csv')
home_page = pd.read_csv('home_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')

In [ ]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [ ]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')

In [ ]:
# Для удобства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


**1.** Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.

In [ ]:
# Подготовим группировку данных по совершению событий на сайте в разрезе полов пользователей
dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()

In [ ]:
# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None

In [ ]:
# Переименуем стандартные после процедуры заголовки
dfg = dfg.rename(columns = {'index':'Page'})

In [ ]:
# Посмотрим, что у нас получилось
dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [ ]:
# Импортируем графическую библиотеку Plotly 
from plotly import graph_objects as go

In [ ]:
# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

In [ ]:
# Часть графика для Male
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Male'],
    name = 'Male',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

In [ ]:
# Часть графика для Female
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Female'],
    name = 'Female',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

In [ ]:
fig.update_layout(
    title = 'Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

**2.** Создать 2 новые фичи на основе колонки “date”: месяц и день недели

In [ ]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['day_of_week'] = pd.DatetimeIndex(df['date']).dayofweek
df.head(15)

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,5
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,3
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,3
5,993454,2015-03-03,Desktop,Male,1,0,0,0,3,1
6,377150,2015-04-15,Desktop,Female,1,1,0,0,4,2
7,137258,2015-01-27,Mobile,Male,1,0,0,0,1,1
8,608249,2015-04-12,Desktop,Female,1,0,0,0,4,6
9,615615,2015-02-24,Mobile,Female,1,1,0,0,2,1


**3.** Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).

In [ ]:
df['month'].loc[df['payment_confirmation']==1].value_counts()

1    189
2    173
4     46
3     44
Name: month, dtype: int64

In [ ]:
df['day_of_week'].loc[df['payment_confirmation']==1].value_counts()

0    77
5    70
1    69
3    66
6    62
2    54
4    54
Name: day_of_week, dtype: int64

**4.** Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).

In [ ]:
dfg = pd.crosstab(df['day_of_week'], df['sex'] )
dfg.columns.name = None
dfg.head(10)

,Female,Male
day_of_week,,
0,6400,6555
1,6410,6287
2,6359,6472
3,6728,6716
4,6328,6422
5,6344,6457
6,6506,6416


In [ ]:
dfg.sort_values(by='Female', ascending = False).head(1)

,Female,Male
day_of_week,,
3,6728,6716


In [ ]:
dfg.sort_values(by='Male', ascending = False).head(1)

,Female,Male
day_of_week,,
3,6728,6716


In [ ]:
dfg.max() # для проверки себя

Female    6728
Male      6716
dtype: int64